<img src="materials/images/introduction-to-genomic-data-cover.png"/>

# Introduction to Genomics

Genomics is the branch of molecular biology concerned with the structure, function, evolution, and mapping of genomes.

In this module, you will be learning about how to process and annotate **Variant Call Format (VCF)** file using Amazon Athena. The Variant Call Format specifies the format of a text file used in bioinformatics for storing gene sequence variations.

The format has been developed with the advent of large-scale genotyping and DNA sequencing projects, such as the ***1000 Genomes Project***.

`🕒 This module should take about 30 minutes to complete.`

`✍️ This notebook is written using Python.`

<div class="alert alert-block alert-info">
<h3>⌨️ Keyboard shortcut</h3>

These common shortcut could save your time going through this notebook:
- Run the current cell: **`Enter + Shift`**.
- Add a cell above the current cell: Press **`A`**.
- Add a cell below the current cell: Press **`B`**.
- Change a code cell to markdown cell: Select the cell, and then press **`M`**.
- Delete a cell: Press **`D`** twice.

Need more help with keyboard shortcut? Press **`H`** to look it up.
</div>



---


In [1]:
# Tables you can query
# ['default.g1000vcf_csv_int', 'default.g1000vcf_csv', 'default.g1000vcf_parquet', 'default.g1000vcf_partitioned']
# COSMIC68 Annotation Dataset ['1000_genomes.hg19_cosmic68_int']
# UCSC RefGene Annotation Dataset ['1000_genomes.hg19_ucsc_refgene_int']

---

# Initial Setup

We'll use the PyAthena library to get access to a database stored in AWS S3. You can read more about PyAthena here:

• https://pypi.org/project/pyathena/

• https://aws.amazon.com/athena/?whats-new-cards.sort-by=item.additionalFields.postDateTime&whats-new-cards.sort-order=desc

In [2]:
import sys
import pyathena
import pandas as pd

from IPython import display

conn = pyathena.connect(s3_staging_dir="s3://athena-output-351869726285/", region_name='us-east-1', encryption_option='SSE_S3')

/tmp/ipykernel_122/317448135.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


---

# Query the 1000 Genomes Project Dataset

It's usually helpful to picture the data before performing any analysis, so we are going to import the database as the first step, and then view a few random rows.

<div class="alert alert-block alert-info">
<b>Tip:</b> If you are new to Jupyter Notebook, try run the code cell below using keyboard shortcut: "Shift" + "Enter". You could look up more keyboard shortcuts by pressing "H".
</div>

In [3]:
# pd.set_option('display.max_colwidth', None) # This code expands the table horizontally so that all table cells are visible.
pd.read_sql('SELECT * FROM default.g1000vcf_csv_int LIMIT 10', conn).head(10)

/tmp/ipykernel_122/653230635.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('SELECT * FROM default.g1000vcf_csv_int LIMIT 10', conn).head(10)


,chrm,start_position,end_position,reference_bases,alternate_bases,rsid,qual,filter,info,chromosome
0,8,115304990,115304991,A,T,rs2132786,100,PASS,AC=3441;AF=0.687101;AN=5008;NS=2504;DP=15751;E...,8
1,8,115347125,115347126,A,T,rs566509966,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=13884;E...,8
2,8,115432741,115432742,A,T,rs150484389,100,PASS,AC=13;AF=0.00259585;AN=5008;NS=2504;DP=18880;E...,8
3,8,115476289,115476290,A,T,rs536578282,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=10646;E...,8
4,8,115523956,115523957,A,T,rs62543474,100,PASS,AC=442;AF=0.0882588;AN=5008;NS=2504;DP=16464;E...,8
5,8,115839770,115839771,A,T,rs554040155,100,PASS,AC=7;AF=0.00139776;AN=5008;NS=2504;DP=16320;EA...,8
6,8,115916679,115916680,A,T,rs78528851,100,PASS,AC=46;AF=0.0091853;AN=5008;NS=2504;DP=18382;EA...,8
7,8,115941532,115941533,A,T,rs6469559,100,PASS,AC=2107;AF=0.420727;AN=5008;NS=2504;DP=13006;E...,8
8,8,115960179,115960180,A,T,rs190204811,100,PASS,AC=12;AF=0.00239617;AN=5008;NS=2504;DP=11854;E...,8
9,8,116205208,116205209,A,T,rs184502983,100,PASS,AC=2;AF=0.000399361;AN=5008;NS=2504;DP=15165;E...,8


We will go through a few important columns for you to understand in the table above. If you want to dive deeper, you may find this document helpful:https://samtools.github.io/hts-specs/VCFv4.2.pdf

1. `chrm` is the chromosome location.
2. `start_position` is the start position of the DNA variant.
3. `end_position` is the end position of the DNA variant.
4. `reference_base` is the allele at a specific location in the reference genomes, which are considered as the "approximated normal".
5. `alternate_base` is the allele that shows up at a specific location in the sample, but does not exist at the corresponding location in the reference genomes.
6. `rsid` is the identification number of the SNPs (Single Nucleotide Polymorphism), see here: https://www.snpedia.com/index.php/SNPedia
7. `qual` is the Phred-scaled quality score for the assertion made in ALT.
8. `filter` indicates PASS when the position has passed all filters.

The `info` column explains additional information about the data. In the example below, we can tell the percentage of East Asian population who has `rs559815820` SNP is only **0.0399361%**. That is 2 alleles count out of 5008 alleles from 2504 samples in the ***1000 Genomes Project***. The percentage of East Asian population who has `rs559815820` SNP is **0.1%**.

<img src="materials/images/example-1.png"/>

Now, try to reference the explanation below about the additional information section. Select a SNP from the data you queried: How does this SNP present in the ethnic group you belong to, in comparison to other ethnic groups?

1. `AC` stands for allele count in genotypes.
2. `AF` stands for allele frequency. Note: allele frequency (AF) = allele count (AC)/ allele number (AN)
3. `AN` stands for total number of alleles in genotypes.
4. `NS` stands for number of samples with data.
5. `EAS_AF` is the allele frequency of **East Asian population**.
6. `AMR_AF` is the allele frequency of **Ad Mixed American population**.
7. `EUR_AF` is the allele frequency of **European population**.
8. `AFR_AF` is the allele frequency of **African population**.
9. `SAS_AF` is the allele frequency of **South Asian population**.

---

# Search for SNPs (Single Nucleotide Polymorphism)

Next, pick a SNPs you are interested in investigating from this website: https://www.snpedia.com

The website has SNPs associated with a wide range of phenotypes. You could start exploring the popular SNPs on the homepage.

Here are a few examples:
1. `rs53576` is highly associated with the ability to empathize with others.
2. `rs72921001` is responsible for certain population's dislike towards the taste of cilantro.
3. `rs28936679` is associated with sleep disorder.
4. `rs1805009` is associated with skin cancer.

The example code below calls the data in the ***1000 Genomes Project*** that has `rs12913832`, see the code  ` WHERE rsid='rs12913832'`.

`rs12913832` is the SNP associated with blue or brown eye color.

In [4]:
pd.set_option('display.max_colwidth', None) # This code expands the table horizontally so that all table cells are visible.
pd.read_sql("SELECT * FROM \"default\".g1000vcf_csv_int WHERE rsid='rs12913832'", conn).head()

/tmp/ipykernel_122/1831946132.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT * FROM \"default\".g1000vcf_csv_int WHERE rsid='rs12913832'", conn).head()


,chrm,start_position,end_position,reference_bases,alternate_bases,rsid,qual,filter,info,chromosome
0,15,28365617,28365618,A,G,rs12913832,100,PASS,AC=888;AF=0.177316;AN=5008;NS=2504;DP=19161;EAS_AF=0.002;AMR_AF=0.2017;AFR_AF=0.028;EUR_AF=0.6362;SAS_AF=0.0706;AA=A|||;VT=SNP,15


Does the result above make sense to you?

We can see allele frequency of `rs12913832` among **Ad Mixed American** and **European** populations are 20.17% and 63.62% respectively, while only 0.2% among **East Asian** and 2.8% among **African** populations.

---

# Query COSMIC68 Annotation Dataset (hg19)

***COSMIC*** database is short for Catalogue of Somatic Mutations in Cancer [hg19 cosmic 68]. Learn more at
https://cancer.sanger.ac.uk/cosmic.

Now, let's take a look at few random rows of the ***COSMIC*** database:

In [5]:
pd.read_sql('SELECT * FROM "1000_genomes".hg19_cosmic68 LIMIT 10', conn).head(10)

/tmp/ipykernel_122/26311468.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('SELECT * FROM "1000_genomes".hg19_cosmic68 LIMIT 10', conn).head(10)


,chrm,start_position,end_position,reference_bases,alternate_bases,cosmic_info
0,7,158380272,158380274,CC,TT,ID=COSM225618;OCCURENCE=1(NS)
1,8,24365021,24365023,CG,TC,ID=COSM1552549;OCCURENCE=1(lung)
2,9,5078357,5078360,TCA,GAG,ID=COSM214344;OCCURENCE=1(haematopoietic_and_lymphoid_tissue)
3,9,32632456,32632458,GG,TT,ID=COSM354681;OCCURENCE=1(lung)
4,20,34775607,34775609,CC,TT,ID=COSM1713410;OCCURENCE=1(skin)
5,20,44678349,44678351,GG,AA,ID=COSM1713574;OCCURENCE=1(skin)
6,20,48231208,48231210,CT,TC,ID=COSN42697;OCCURENCE=1(central_nervous_system)
7,9,71094387,71094389,CC,TT,ID=COSM1701102;OCCURENCE=1(skin)
8,9,98221989,98221991,CC,TT,ID=COSM14463;OCCURENCE=2(skin)
9,21,37833905,37833907,AG,TT,ID=COSM310129;OCCURENCE=1(lung)


Like previous datasets, we see the chromosome location, start and end positions in DNA sequence, reference allele and alternate allele.

The `cosmic_info` column tells us the types of cancer and mutation occurence.

---

# Query UCSC RefGene Annotation Dataset (hg19)

The NCBI RefSeq Genes composite track shows human protein-coding and non-protein-coding genes taken from the NCBI RNA reference sequences collection (RefSeq) [hg19 refGene]. You could learn more at the following:
- https://hgdownload.cse.ucsc.edu/goldenPath/hg19/database/
- https://genome.ucsc.edu/cgi-bin/hgTables?db=hg19&hgta_group=genes&hgta_track=refSeqComposite&hgta_table=refGene&hgta_doSchema=describe+table+schema (Schema for NCBI RefSeq - RefSeq genes from NCBI)

<img src="materials/images/genetic-mutations.png"/>

Now, let's take a look at random rows of the dataset:

In [6]:
pd.read_sql('SELECT * FROM "1000_genomes".hg19_ucsc_refgene_int LIMIT 10', conn).head(10)

/tmp/ipykernel_122/730593987.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql('SELECT * FROM "1000_genomes".hg19_ucsc_refgene_int LIMIT 10', conn).head(10)


,chrm,start_position,end_position,bin,name,strand,cdsstart,cdsend,exoncount,exonstarts,exonends,score,name2,cdsstartstat,cdsendstat,exonframes
0,8,48773459,48773532,119,NM_006904,-,48686733,48872686,86,"48685668,48689404,48690246,48691019,48691288,48691564,48694722,48694938,48696302,48697673,48701466,48701711,48706850,48710797,48711770,48713353,48715866,48719697,48730010,48731962,48733279,48734164,48736418,48739216,48740728,48743165,48744374,48746756,48748898,48749772,48751708,48752576,48761714,48761939,48765233,48766643,48767782,48769716,48771076,48771409,48772171,48773459,48774622,48774933,48775959,48777116,48790284,48792051,48793976,48794472,48798504,48800107,48801078,48801574,48802817,48805699,48809720,48811029,48812932,48815128,48817428,48824969,48826460,48827887,48830836,48839753,48840330,48841651,48842412,48843231,48845579,48846524,48847568,48848291,48848912,48852110,48855768,48856412,48856533,48866179,48866366,48866897,48868433,48869730,48869914,48872532,","48686938,48689544,48690435,48691221,48691360,48691654,48694815,48695159,48696370,48697878,48701610,48701799,48707062,48710958,48711951,48713547,48716041,48719887,48730122,48732071,48733504,48734353,48736557,48739422,48740908,48743297,48744487,48746957,48749088,48749980,48751807,48752750,48761864,48762064,48765345,48766775,48767934,48769860,48771196,48771547,48772320,48773532,48774688,48775102,48776138,48777324,48790412,48792219,48794081,48794658,48798708,48800266,48801211,48801783,48803041,48805947,48809854,48811129,48813027,48815355,48817536,48825122,48826624,48827978,48830943,48839913,48840450,48841738,48842572,48843347,48845732,48846650,48847618,48848460,48849077,48852257,48855926,48856443,48856589,48866279,48866479,48867006,48868508,48869823,48869991,48872743,",0,PRKDC,cmpl,cmpl,"2,0,0,2,2,2,2,0,1,0,0,2,0,1,0,1,0,2,1,0,0,0,2,0,0,0,1,1,0,2,2,2,2,0,2,2,0,0,0,0,1,0,0,2,0,2,0,0,0,0,0,0,2,0,1,1,2,1,2,0,0,0,1,0,1,0,0,0,2,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,"
1,8,48774622,48774688,119,NM_006904,-,48686733,48872686,86,"48685668,48689404,48690246,48691019,48691288,48691564,48694722,48694938,48696302,48697673,48701466,48701711,48706850,48710797,48711770,48713353,48715866,48719697,48730010,48731962,48733279,48734164,48736418,48739216,48740728,48743165,48744374,48746756,48748898,48749772,48751708,48752576,48761714,48761939,48765233,48766643,48767782,48769716,48771076,48771409,48772171,48773459,48774622,48774933,48775959,48777116,48790284,48792051,48793976,48794472,48798504,48800107,48801078,48801574,48802817,48805699,48809720,48811029,48812932,48815128,48817428,48824969,48826460,48827887,48830836,48839753,48840330,48841651,48842412,48843231,48845579,48846524,48847568,48848291,48848912,48852110,48855768,48856412,48856533,48866179,48866366,48866897,48868433,48869730,48869914,48872532,","48686938,48689544,48690435,48691221,48691360,48691654,48694815,48695159,48696370,48697878,48701610,48701799,48707062,48710958,48711951,48713547,48716041,48719887,48730122,48732071,48733504,48734353,48736557,48739422,48740908,48743297,48744487,48746957,48749088,48749980,48751807,48752750,48761864,48762064,48765345,48766775,48767934,48769860,48771196,48771547,48772320,48773532,48774688,48775102,48776138,48777324,48790412,48792219,48794081,48794658,48798708,48800266,48801211,48801783,48803041,48805947,48809854,48811129,48813027,48815355,48817536,48825122,48826624,48827978,48830943,48839913,48840450,48841738,48842572,48843347,48845732,48846650,48847618,48848460,48849077,48852257,48855926,48856443,48856589,48866279,48866479,48867006,48868508,48869823,48869991,48872743,",0,PRKDC,cmpl,cmpl,"2,0,0,2,2,2,2,0,1,0,0,2,0,1,0,1,0,2,1,0,0,0,2,0,0,0,1,1,0,2,2,2,2,0,2,2,0,0,0,0,1,0,0,2,0,2,0,0,0,0,0,0,2,0,1,1,2,1,2,0,0,0,1,0,1,0,0,0,2,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,"
2,8,48790284,48790412,119,NM_006904,-,48686733,48872686,86,"48685668,48689404,48690246,48691019,48691288,48691564,48694722,48694938,48696302,48697673,48701466,48701711,48706850,48710797,48711770,48713353,48715866,48719697,48730010,48731962,48733

Here are what some of the columns mean according to Schema for NCBI RefSeq - RefSeq genes from NCBI:

1. `cdsstart`: Coding region start.
2. `cdsend`: Coding region end.
3. `exoncount`: Number of exons.
4. `strand`: + or - for strand


---

# Variant-Based Annotation

Variant-based annotation aims to look for **exact matches** between a query variant and a record in annotation datasets (i.e., two items have identical  chromosome, start position, end position, reference allele and alternative allele).

The code below uses the `JOIN` function to look for exact matches between the ***1000 Genomes Project*** dataset and the ***COSMIC*** dataset. It compares the start and end positions, reference allele, alternate allele at chromosome 2 between the two datasets.

In [7]:
pd.read_sql("SELECT A.chrm, A.start_position, A.end_position, A.reference_bases, A.alternate_bases,B.cosmic_info, A.info "
+ " FROM (SELECT * FROM \"default\".g1000vcf_csv_int WHERE chrm='2') as A "
+ " JOIN "
+ " (SELECT * FROM \"1000_genomes\".hg19_cosmic68_int WHERE chrm='2') as B "
+ " ON A.start_position=B.start_position AND A.alternate_bases=B.alternate_bases "
+ " ORDER By  A.start_position", conn).head()

/tmp/ipykernel_122/4159624003.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT A.chrm, A.start_position, A.end_position, A.reference_bases, A.alternate_bases,B.cosmic_info, A.info "


,chrm,start_position,end_position,reference_bases,alternate_bases,cosmic_info,info
0,2,55184,55185,G,A,ID=COSN206912;OCCURENCE=1(large_intestine),AC=60;AF=0.0119808;AN=5008;NS=2504;DP=19415;EAS_AF=0;AMR_AF=0.0043;AFR_AF=0.0431;EUR_AF=0;SAS_AF=0;AA=G|||;VT=SNP
1,2,95490,95491,C,T,ID=COSN42681;OCCURENCE=4(central_nervous_system),AC=17;AF=0.00339457;AN=5008;NS=2504;DP=16854;EAS_AF=0.0169;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=C|||;VT=SNP
2,2,128457,128458,G,A,"ID=COSN181337,COSN181637;OCCURENCE=1(stomach),2(large_intestine)",AC=1;AF=0.000199681;AN=5008;NS=2504;DP=17605;EAS_AF=0.001;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=G|||;VT=SNP
3,2,133636,133637,G,A,"ID=COSN33464;OCCURENCE=1(breast),1(large_intestine)",AC=3;AF=0.000599042;AN=5008;NS=2504;DP=19368;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0;SAS_AF=0.0031;AA=G|||;VT=SNP
4,2,218857,218858,G,A,"ID=COSM1404910,COSM1404911;OCCURENCE=1(large_intestine)",AC=1;AF=0.000199681;AN=5008;NS=2504;DP=17149;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.001;SAS_AF=0;AA=G|||;VT=SNP;EX_TARGET


The returned table tells us information about genes associated with different types of cancers.

In the example below, the cancer related to central nervous system is at chromosome 2, where the reference allele is C and alternate allele is T. It appears to only happen to **East Asian** population with a frequency of 1.69% while absent among other populations.

<img src="materials/images/example-2.png"/>

---

# Interval-Based Annotation [TP53: chrm17]

The aim of interval-based annotation is to look for overlap of a query variant with a region (this region could be a single position) in annotation databases.

<img src="materials/images/overlapping-condition.png"/>

<div class="alert alert-block alert-info">
<b> Green.start_position[X]<=Blue.end_position[B] AND Blue.start_position[A]<=Green.end_position[Y]</b>

Source: https://stackoverflow.com/questions/20981783/how-to-sum-overlapping-interval-parts-in-sql.
</div>

The code below uses the `JOIN` function to compare two datasets using overlapping condition.

1. Rather than comparing whether the two datasets are exactly the same, this method focuses on overlapping regions. In the following example, we are running chromosome 17 from the ***1000 Genomes Project*** against gene TP53 from chromosome 17 in the ***UCSC RefGene Annotation*** dataset.
2. The `ON` condition is trying to see if there is any overlapping between the two datasets in the selected region. See the graph below:


In [8]:
pd.read_sql("SELECT A.chrm, A.start_position, A.end_position, A.reference_bases, A.alternate_bases,B.name, B.name2, A.info "
+ " FROM (SELECT * FROM \"default\".g1000vcf_csv_int WHERE chrm='17') as A "
+ " JOIN "
+ " (SELECT * FROM \"1000_genomes\".hg19_ucsc_refgene_int WHERE chrm='17' and name2='TP53') as B "
+ " ON A.start_position<=B.end_position AND B.start_position<=A.end_position "
+ " ORDER By  A.start_position", conn).head()

/tmp/ipykernel_122/1411853846.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("SELECT A.chrm, A.start_position, A.end_position, A.reference_bases, A.alternate_bases,B.name, B.name2, A.info "


,chrm,start_position,end_position,reference_bases,alternate_bases,name,name2,info
0,17,7571751,7571752,T,G,NM_001276697,TP53,AC=13;AF=0.00259585;AN=5008;NS=2504;DP=19133;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.0129;SAS_AF=0;AA=T|||;VT=SNP
1,17,7571751,7571752,T,G,NM_001126115,TP53,AC=13;AF=0.00259585;AN=5008;NS=2504;DP=19133;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.0129;SAS_AF=0;AA=T|||;VT=SNP
2,17,7571751,7571752,T,G,NM_001126117,TP53,AC=13;AF=0.00259585;AN=5008;NS=2504;DP=19133;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.0129;SAS_AF=0;AA=T|||;VT=SNP
3,17,7571751,7571752,T,G,NM_001276699,TP53,AC=13;AF=0.00259585;AN=5008;NS=2504;DP=19133;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.0129;SAS_AF=0;AA=T|||;VT=SNP
4,17,7571751,7571752,T,G,NM_001126116,TP53,AC=13;AF=0.00259585;AN=5008;NS=2504;DP=19133;EAS_AF=0;AMR_AF=0;AFR_AF=0;EUR_AF=0.0129;SAS_AF=0;AA=T|||;VT=SNP


<img src="materials/images/conductor.png"/>

For your information, TP53 gene provides instructions for making a protein called tumor protein p53 (or p53). This protein acts as a tumor suppressor, which means that it regulates cell division by keeping cells from growing and dividing (proliferating) too fast, or in an uncontrolled way. TP53 acts like a conductor in an orchestra.

The table gives us a view of all the mutations on gene TP53 on chromosome 2. Typically, mutations on TP53 are considered as rare mutations.

---

# Contributions & acknowledgement

Thank the following team for working on this module:

- **Module Content**: Amir Bahmani
- **Engineering**: Amit Dixit
- **UX/UI Design & Illustration**: Kexin Cha
- **Video Production**: Francesca Goncalves
- **Project Management**: Amir Bahmani, Kexin Cha

---

Copyright (c) 2022 Stanford Data Ocean (SDO)

All rights reserved.